<a href="https://colab.research.google.com/github/nirmal129/Battle-of-Millenniums/blob/master/Battle_of_Millenniums.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and Imports

In [ ]:
%pip install qiskit --quiet
%pip install pylatexenc --quiet

In [ ]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.quantum_info import Statevector
import numpy as np
import random
from qiskit.visualization import plot_histogram, plot_state_qsphere, plot_bloch_multivector, plot_bloch_vector
from qiskit.circuit.random import random_circuit

In [ ]:
def qft(qc, q):
    n = len(q)
    for j in range(n-1, -1, -1):
        qc.h(q[j])
        for k in range(j-1, -1, -1):
            qc.cu1(2*np.pi*2**(-(j-k+1)), q[k], q[j])  
    for l in range(int(np.floor(n/2))):
        qc.swap(q[l], q[n-1-l])

def iqft(qc, q):
    n = len(q)
    for l in range(int(np.floor(n/2))):
        qc.swap(q[l], q[n-1-l])
    for j in range(n):
        for k in range(j):
            qc.cu1(-2*np.pi*2**(-(j-k+1)), q[k], q[j])
        qc.h(q[j])

In [ ]:
def multiFlip(qc, q_controls, q_target):
    qc.h(q_target)
    multiPhase(qc, q_controls+[q_target], np.pi)
    qc.h(q_target)

def multiPhase(qc, q, theta):
    if len(q) == 1:
        qc.u1(theta, q[0])
    elif len(q) == 2:
        qc.cu1(theta, q[0], q[1])
    else:
        qc.cu1(theta/2, q[1], q[0])        
        multiFlip(qc, q[2:], q[1])
        qc.cu1(-theta/2, q[1], q[0])
        multiFlip(qc, q[2:], q[1])
        multiPhase(qc, [q[0]]+q[2:], theta/2)

# Bonuses

### Arithmetic Bonuses

In [ ]:
# Qubit addition operator

def add(qc, qs_input, qs_output):
  qft(qc, qs_output)
  m = len(qs_output)
  n = len(qs_input)
  for i in range(m):
    for j in range(0, m - i):
      qc.crz(2 * np.pi / (2 ** (m - i - j)), qs_input[j], qs_output[i])
  iqft(qc, qs_output)

In [ ]:
# Qubit subtraction operator

def sub(qc, qs_input, qs_output):
  qft(qc, qs_output)
  m = len(qs_output)
  n = len(qs_input)
  for i in range(m):
    for j in range(0, m - i):
      qc.crz(-2 * np.pi / (2 ** (m - i - j)), qs_input[j], qs_output[i])
  iqft(qc, qs_output)

In [ ]:
# Qubit multiplication operator 

def mul(qc, qx, qy, qa):
    qft(qc, qa)
    n = len(qx)
    m = len(qy)
    p = len(qa)
    for i in range(n):
      for j in range(m):
        for k in range(p - j - i):
          li = [qa[k], qy[j], qx[i]]
          theta = 2 * np.pi / (2 ** (p - k - j - i))
          multiPhase(qc, li, theta)   
    iqft(qc, qa)

### Grover's Bonus

In [ ]:
def U(qc, q, n):
  circ = random_circuit(n, n)
  qc += circ

In [ ]:
def W(qc, q, n):
  qc.h(q[1:])
  qc.x(q[1:])
  multiPhase(qc, q, np.pi / 2)
  qc.x(q[1:])
  qc.h(q[1:])

In [ ]:
# Grover algorithm on randomized circuit with n - 1 qubits

def GroverS(n):
  q = QuantumRegister(n)
  c = ClassicalRegister(n - 1)
  qc = QuantumCircuit(q, c)
  R = np.floor(np.pi * np.sqrt(2 ** n) / 4)
  qc.x(q[0])
  qc.h(q[0])
  qc.h(q[1:])
  for i in range(int(R)):
    U(qc, q, n)
    W(qc, q, n)
  qc.h(q[0])
  qc.x(q[0])
  qc.measure(q[1:], c)
  backend = Aer.get_backend('qasm_simulator')
  counts= execute(qc,backend, shots=1024).result().get_counts(qc)
  return int(max(counts, key = counts.get), 2)

### Deutsch's Assignment

In [ ]:
# Deutsch algorithm to decide team
# Take name (String) as an argument
# The name is converted into bit string by appending the modulus 2 of ASCII value of character's in name
# Applies (C_n0^cn)*(C_(n-1)0^c(n-1))*...*(C_10^c1)*(X_0^c0) where ci is ith bit of bit string generated from previous step
# Returns token which is 0 if the max returned result is 0 else 1.

def Deutsch(name):
  bitString = ""
  for c in name:
    bitString += str(ord(c) % 2)
  n = len(name)
  q = QuantumRegister(n + 1)
  c = ClassicalRegister(n)
  qc = QuantumCircuit(q, c)
  qc.x(q[0])
  qc.h(q)
  if bitString[0] == '1':
    qc.x(q[0])
  for i in range(1, n):
    if bitString[i] == '1':
      qc.cx(q[i], q[0])
  qc.h(q)
  qc.measure(q[1:], c)
  backend = Aer.get_backend('qasm_simulator')
  counts = execute(qc,backend, shots=1024).result().get_counts(qc)
  token = int(max(counts, key = counts.get), 2)
  if token != 0:
    token = 1
  return token

# Game

### Outline

Player will enter his/her/their name. Deutsch alogrithm will decide whether the player will join team of angels or devils. According to the team and name, the player will receive skills (Offensive, defensive, and healing). 

The battle will be of two players. There are two types of bonuses for battle: Arithmetic bonuses and Grover's bonus.

For the arithmetic add bonus, the player offensive power will increase by making a combo of two/three attacks. For the arithmetic sub bonus, the player's defensive power will increase. Add and Sub are both of 3 qubits. For the arithmetic mul bonus, the player will deal additional damage with the price of 4 magic points.

For the Grover's bonus, Grover's search algorithm will be run on a random gate. If the returned search is less than 12 then bit flip will be applied. Grover is of 5 qubits. The price for Grover's bonus is 8 magic points.

Initial health will be 100 points. The magic capacity will be 10 points. For every attack, opposite player's magic will be filled by 20% for Grover's bonus attack points and 30% for normal attack points.

Add and Sub bonuses are random. Mul bonus requires 4 magic points and Grover's bonus requires 8 magic points.

### Supplementary

In [ ]:
# Get attack, heal for player's choice

def move(choice):
  attack = heal = 0

  # Initial setup
  if choice == 2:
    attack += 4
  elif choice == 3:
    attack += 6
  else:
    heal += 6
  
  # Random circuit with superposition between attack and heal qubits
  
  qc = random_circuit(5, 4, measure = True)
  qc.h(range(5))
  backend = Aer.get_backend('qasm_simulator')
  counts = execute(qc,backend, shots=1024).result().get_counts(qc)
  measure = max(counts, key = counts.get)
  sec1 = str(measure)[0 : 3]
  sec2 = str(measure)[3 : 5]
  if choice == 4:
    heal += int(sec1, 2)
    attack += int(sec2, 2)
  else:
    attack += int(sec1, 2)
    heal += int(sec2, 2)
  return attack, heal

In [ ]:
# Get bonus's attack, heal, and enhance (Enhance only for Sub bonus)

def bonuses(bonus):

  attackB = healB = 0
  enhance = 1
  
  # Mul bonus
  # Requires 4 magic points
  # 4 qubit attack (Max bonus attack 2 ** 4 = 16)

  if bonus == "Mul":
    magic[chance] -= 4

    # Random XZ circuit with superposition

    binx = [getBinString(4), getBinString(4), getBinString(4), getBinString(4)]
    biny = [getBinString(4), getBinString(4), getBinString(4), getBinString(4)]
    qx = QuantumRegister(4)
    qy = QuantumRegister(4)
    qa = QuantumRegister(4)
    c = ClassicalRegister(4)
    qc = QuantumCircuit(qx, qy, qa, c)
    qc.h(qx)
    qc.h(qy)
    for i in range(4):
      for j in range(4):
        if binx[i][j] == '0':
          qc.x(qx[i])
        else:
          qc.z(qx[i])
        if biny[i][j] == '0':
          qc.x(qy[i])
        else:
          qc.z(qy[i])
    mul(qc, qx, qy, qa)
    qc.measure(qa, c)
    backend = Aer.get_backend('qasm_simulator')
    counts = execute(qc,backend, shots=1024).result().get_counts(qc)
    attackB = int(max(counts, key = counts.get), 2)
    return attackB, healB, enhance
  
  # Add bonus
  # No magic points required
  # 3 qubit attack (Max bonus attack 2 ** 3 = 8)

  elif bonus == "Add":

    # Random XZ circuit with superposition

    bini = [getBinString(3), getBinString(3), getBinString(3)]
    bino = [getBinString(3), getBinString(3), getBinString(3)]
    qi = QuantumRegister(3)
    qo = QuantumRegister(3)
    c = ClassicalRegister(3)
    qc = QuantumCircuit(qi, qo, c)
    qc.h(qi)
    for i in range(3):
      for j in range(3):
        if bini[i][j] == '0':
          qc.x(qi[i])
        else:
          qc.z(qi[i])
        if bino[i][j] == '0':
          qc.x(qo[i])
        else:
          qc.z(qo[i])
    add(qc, qi, qo)
    qc.measure(qo, c)
    backend = Aer.get_backend('qasm_simulator')
    counts = execute(qc,backend, shots=1024).result().get_counts(qc)
    attackB = int(max(counts, key = counts.get), 2)
    return attackB, healB, enhance
  
  # Add bonus
  # No magic points required
  # 3 qubit fractional enhance
  # Max bonus enhance = 0 meaning enemy's attack will deal no damage
  # Min bonus enhance = 1 meaning enemy's attack will not be altered

  elif bonus == "Sub":

    # Random XZ circuit with superposition

    bini = [getBinString(3), getBinString(3), getBinString(3)]
    bino = [getBinString(3), getBinString(3), getBinString(3)]
    qi = QuantumRegister(3)
    qo = QuantumRegister(3)
    c = ClassicalRegister(3)
    qc = QuantumCircuit(qi, qo, c)
    qc.h(qi)
    for i in range(3):
      for j in range(3):
        if bini[i][j] == '0':
          qc.x(qi[i])
        else:
          qc.z(qi[i])
        if bino[i][j] == '0':
          qc.x(qo[i])
        else:
          qc.z(qo[i])
    sub(qc, qi, qo)
    qc.measure(qo, c)
    backend = Aer.get_backend('qasm_simulator')
    counts = execute(qc,backend, shots=1024).result().get_counts(qc)
    enhance = int(max(counts, key = counts.get), 2)
    enhance = enhance / 8
    return attackB, healB, enhance

In [ ]:
# Random bit string generator

def getBinString(n):
  q = QuantumRegister(n)
  c = ClassicalRegister(n)
  qc = QuantumCircuit(q, c)
  qc.h(q)
  qc.measure(q, c)
  backend = Aer.get_backend('qasm_simulator')
  counts = execute(qc,backend, shots=1024).result().get_counts(qc)
  measure = max(counts, key = counts.get)
  return measure

### Implementation

In [ ]:
# Get names

name1 = input("Player 1 enter your name: ")
name2 = input("Player 2 enter your name: ")
while len(name1) == 0 or len(name2) == 0:
  print("Please enter valid name")
  name1 = input("Player 1 enter your name: ")
  name2 = input("Player 2 enter your name: ")

name = [name1, name2]
token1 = Deutsch(name1)
token2 = Deutsch(name2)

# Deciding team

teamE = ["\U0001F607", "\U0001F608"]
print()
for i in range(16):
  print("\U0001F525", end = "")
print()
print("Lord Deutsch will decide your fate!!")
for i in range(16):
  print("\U0001F525", end = "")
print()
print()
player1T = 0
player2T = 1
if token1 ^ token2 == 1:
  player1T = 1
  player2T = 0

# Print the team assignments

team = [player1T, player2T] # team index for attacks
print("Team", name1, ":\t", teamE[player1T])
print("Team", name2, ":\t", teamE[player2T])

In [ ]:
# Initialization
# Should be run before every game

health = [100, 100]
magic = [0, 0]
defense = [1.0, 1.0]

aAttacks = ["Divine slash (Min attack: 0)", "Judgement ray (Min attack: 4)", "Grand cross (Min attack: 6)", "Holy water (Min heal: 6)", "Touch of Jesus (Min attack: 12) (Requires 8 magic points)"]
dAttacks = ["Black fist (Min attack: 0)", "Flame pillar (Min attack: 4)", "Broken cross (Min attack: 6)", "Lucifier's blood (Min heal: 6)", "Scream of Satan (Min attack: 12) (Requires 8 magic points)"]

attacks = [aAttacks, dAttacks]

In [ ]:
# Game

chance = 0

print("=====Game begins!=====Game begins!=====Game begins!=====Game begins!=====Game begins!=====")
print()

# Chances

while health[0] > 0 and health[1] > 0:

  # Print stats
  
  print(name[chance] + "! your turn")
  print()
  print("Health:\t", health[chance])
  print("Magic:\t", magic[chance])
  print()
  print("Enemy's stats")
  print()
  print("Health:\t", health[chance ^ 1])
  print("Magic:\t", magic[chance ^ 1])
  print()

  # Prompt current player for move

  print("\tMoves")
  for i in range(5):
    print("\t", i + 1, ")", attacks[team[chance]][i])
  choice = int(input("\tEnter your choice (1 - 5): "))
  while (choice < 1 or choice > 5) or (choice == 5 and magic[chance] < 8):
    if choice == 5 and magic[chance] < 8:
      print("\tNot enough magic")
    else:
      print("\tEnter a valid choice")
    choice = int(input("\tEnter your choice (1 - 5): "))
  print()
  print("\tYour choice: ", attacks[team[chance]][choice - 1])
  print()

  # Computing attack, heal, and enhance

  enhance = 1

  attack = heal = 0

  gUsed = False

  if choice != 5:
    attack, heal = move(choice)
    attackB = healB = 0
    enhance = 1

    # Bonus spinner

    spin = random.uniform(0, 1)
    bonus = ""
    if 0.7 < spin < 0.8:
      bonus = "Add"
    elif 0.8 < spin < 0.9:
      bonus = "Sub"
    elif 0.9 < spin < 1:
      bonus = "Mul"
    if (bonus == "Mul" and magic[chance] >= 4) or (bonus != "Mul" and bonus != ""):
      print("\tYou received a bonus!!")
      if bonus == "Mul":
        print("\tYour bonus: ", bonus, "(Requires 4 magic points)")
        use = input("\tWant to use bonus? (Y for yes else no)")
        if use == "Y":
          attackB, healB, enhance = bonuses(bonus)
      else:
        print("\tYour bonus: ", bonus)
        use = input("\tWant to use bonus? (Y for yes else no) ")
        if use == "Y":
          attackB, healB, enhance = bonuses(bonus)
      print()
  else:
    gUsed = True
    magic[chance] -= 8
    attack = GroverS(6)
    if attack < 12:
      attack = attack ^ (2 ** 6 - 1)
  
  print("Attack: " + str(attack))
  print("Heal: " + str(heal))
  print()
  
  # Capping attack, health, and magic

  if health[chance ^ 1] - int(defense[chance ^ 1] * attack) < 0:
    health[chance ^ 1] = 0
  else:
    health[chance ^ 1] = health[chance ^ 1] - int(defense[chance ^ 1] * attack)
  if health[chance] + heal > 100:
    health[chance] = 100
  else:
    health[chance] = health[chance] + heal
  addMagic = 0
  if gUsed: # Check if Grover is used or not to add magic accordingly
    addMagic = int(attack * 0.2)
  else:
    addMagic = int(attack * 0.3)
  if magic[chance ^ 1] + addMagic >= 10:
    magic[chance ^ 1] = 10
  else:
    magic[chance ^ 1] += addMagic
  defense[chance] = enhance

  chance = chance ^ 1

print("=====Game Over!=====Game Over!=====Game Over!=====Game Over!=====Game Over!=====")

In [ ]:
# Print results

print("Results!")
print()

if health[0] <= 0:
  print("Winner is " + teamE[team[1]] + " aka " + name[1])
else:
  print("Winner is " + teamE[team[0]] + " aka " + name[0])